In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import pylab as plt
%pylab inline

from scipy.signal import resample_poly

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Read in data
path = 'Activity Recognition exp/Activity Recognition exp/Phones_accelerometer.csv'
paccel = pd.read_csv(path, index_col=0)

paccel.tail(8)

,Arrival_Time,Creation_Time,x,y,z,User,Model,Device,gt
Index,,,,,,,,,
129045,1424778553265,92263731712000,1.379043,0.306454,9.959755,i,samsungold,samsungold_2,bike
129046,1424778553286,92263752250000,1.532270,0.153227,9.959755,i,samsungold,samsungold_2,bike
129047,1424778553306,92263772270000,1.379043,0.000000,9.959755,i,samsungold,samsungold_2,bike
129048,1424778553315,92263781761000,1.379043,0.153227,9.959755,i,samsungold,samsungold_2,bike
129049,1424778553346,92263812248000,1.379043,0.153227,9.806528,i,samsungold,samsungold_2,bike
129050,1424778553366,92263832267000,1.532270,0.153227,9.806528,i,samsungold,samsungold_2,bike
129051,1424778553386,92263852409000,1.532270,0.000000,9.959755,i,samsungold,samsungold_2,bike
129052,1424778553395,92263861839000,1.379043,0.000000,9.959755,i,samsungold,samsungold_2,bike


In [7]:
#Resample all trials to 100Hz
x_fixed = paccel.reset_index().groupby(['User','Model','Device','gt'])['x'].apply(lambda x : resample_poly(x.values, 15000, x.values.shape[0]))
y_fixed = paccel.reset_index().groupby(['User','Model','Device','gt'])['y'].apply(lambda y : resample_poly(y.values, 15000, y.values.shape[0]))
z_fixed = paccel.reset_index().groupby(['User','Model','Device','gt'])['z'].apply(lambda z : resample_poly(z.values, 15000, z.values.shape[0]))

#Re-construct
new_df = x_fixed.to_frame()
new_df = new_df.merge(y_fixed.to_frame(), left_index=True, right_index=True)
new_df = new_df.merge(z_fixed.to_frame(), left_index=True, right_index=True)

In [65]:
#Standardize each series, and apply sliding window with length 100 and 50% overlap

#sig = lambda x : 1/(1 + np.exp(-x)) # sigmoid function

images = []
for i in range(new_df.shape[0]):
    x = new_df.iloc[[i]].x.values[0]
    y = new_df.iloc[[i]].y.values[0]
    z = new_df.iloc[[i]].z.values[0]

    x = ((x - x.mean()) / x.std())
    y = ((y - y.mean()) / y.std())
    z = ((z - z.mean()) / z.std())
    
    r = np.stack([x[i:100+i] for i in range(0, 14950, 50)]) # stack elements using 100-sample window with 50% overlap
    g = np.stack([y[i:100+i] for i in range(0, 14950, 50)])
    b = np.stack([z[i:100+i] for i in range(0, 14950, 50)])
    
    vals = np.dstack([r,g,b])
    images.append(vals)
    
im_arr = np.array(images)

In [67]:
im_arr.shape

(485, 299, 100, 3)

In [66]:
im_arr[0]

array([[[-0.31969175,  1.54587967, -5.53462403],
        [-2.81318672,  2.42132793, -2.65935226],
        [-2.0037575 ,  1.9087304 , -3.75544795],
        ...,
        [-2.15607704,  2.14214743, -2.71059801],
        [-2.28155033,  2.01158073, -2.80924805],
        [-2.24770779,  1.83332006, -2.88060289]],

       [[-1.76631062,  4.38346196, -1.07517941],
        [-1.77056579,  3.72143745, -1.06267993],
        [-1.44315492,  3.30233537, -1.3577314 ],
        ...,
        [-3.73712589, -1.01013166, -3.24314148],
        [-3.68208529, -0.72546475, -3.38413602],
        [-3.43814237, -0.45934694, -3.17348672]],

       [[-2.23066986,  1.53558617, -3.09697615],
        [-1.53503089,  1.28817752, -3.38572827],
        [-1.73345454,  0.85084826, -3.57522508],
        ...,
        [-1.85424213,  0.08600117, -3.74413206],
        [-1.7913919 ,  0.17909599, -3.69485741],
        [-1.58206471,  0.17635372, -3.78434762]],

       ...,

       [[-0.49424409, -0.75890629, -0.70153929],
        [-0

In [59]:
a = np.load('image_labels.npy')

cats = pd.Categorical(a).codes

In [60]:
cats

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 3, 5, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
       2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
       6, 2, 3, 4, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 6, 0, 1, 2, 3, 4, 5, 6,
       0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,

In [63]:
np.save('image_array.npy', im_arr)